# MIRI PSF Photometry With Photutils

**Author**: Ori Fox<br>

**Submitted**: November, 2023<br>
**Updated**: November, 2023<br>

**Use case**: PSF Photometry using dedicated package photutils (https://photutils.readthedocs.io/en/stable/). The purpose here is to illustrate the workflow and runtime for using photutils in a variety of use cases. Generally, PSF photometry for data from a space telescope is most accurately performed on pre-mosaiced data. In the case of HST, that corresponds to FLT files rather than DRZ. And in the case of JWST, this corresponds to Level2 files rather than Level3. The reason is that a mosaiced PSF changes the inherent PSF as a function of position on the detector so that there is no adequate model (theoretical or empirical) to use. **NOTE:** A companion notebook exists that illustrates how to use perform PSF photometry on both Level2 and Level3 data using a new software program called space_phot.<br>
**Data**: MIRI Data PID 1028 (Calibration Program; Single Star Visit 006 A5V dwarf 2MASSJ17430448+6655015) and MIRI Data PID 1171 (LMC; Multiple Stars).<br>
**Tools**: photutils, drizzlepac, jupyter <br>
**Cross-Instrument**: NIRCam, MIRI.<br>
**Documentation**: This notebook is part of a STScI's larger post-pipeline Data Analysis Tools Ecosystem and can be downloaded directly from the JDAT Notebook Github directory.<br>
**Pipeline Version**: JWST Pipeline<br>


## Table of contents
1. [Introduction](#intro)<br>
    1.1 [Setup](#webbpsf)<br>
    1.2 [Python imports](#py_imports)<br>
2. [Download Data](#data)<br>
3. [Bright, Single Object](#bso)<br>
    3.1 [Multiple, Level2 Files](#bso2)<br>
4. [Faint/Upper Limit, Single Object](#fso)<br>
    4.1 [Multiple, Level2 Files](#fso2)<br>
5. [Stellar Field (LMC)](#lmv)<br>
    5.1 [Multiple, Level2 Files](#lmc2)<br>

1.<font color='white'>-</font>Introduction <a class="anchor" id="intro"></a>
------------------

GOALS:<br>

PSF Photometry can be obtained using:<br>

* grid of PSF models from WebbPSF<br>
* single effective PSF (ePSF) NOT YET AVAILABLE<br>
* grid of effective PSF NOT YET AVAILABLE<br>

The notebook shows:<br>

* how to obtain the PSF model from WebbPSF (or build an ePSF)<br>
* how to perform PSF photometry on the image<br>

**Data**:<br>

MIRI Data PID 1028 (Calibration Program), F770W <br>
MIRI Data PID 1171 (LMC), F560W/F770W

In [ ]:
%matplotlib inline

### 1.1<font color='white'>-</font>Setup WebbPSF and Synphot Directories<a class="anchor" id="webbpsf"></a> ###

In [ ]:
%matplotlib inline

In [ ]:
import sys,os,glob,shutil
import tarfile, urllib.request

# Set environmental variables
os.environ["WEBBPSF_PATH"] = "./webbpsf-data/webbpsf-data"
os.environ["PYSYN_CDBS"] = "./grp/redcat/trds/"

# WEBBPSF Data
boxlink = 'https://stsci.box.com/shared/static/qxpiaxsjwo15ml6m4pkhtk36c9jgj70k.gz'                                                           
boxfile = './webbpsf-data/webbpsf-data-1.0.0.tar.gz'
synphot_url = 'http://ssb.stsci.edu/trds/tarfiles/synphot5.tar.gz'
synphot_file = './synphot5.tar.gz'

webbpsf_folder = './webbpsf-data'
synphot_folder = './grp'

# Gather webbpsf files
psfExist = os.path.exists(webbpsf_folder)
if not psfExist:
    os.makedirs(webbpsf_folder)
    urllib.request.urlretrieve(boxlink, boxfile)
    gzf = tarfile.open(boxfile)
    gzf.extractall(webbpsf_folder)

# Gather synphot files
synExist = os.path.exists(synphot_folder)
if not synExist:
    os.makedirs(synphot_folder)
    urllib.request.urlretrieve(synphot_url, synphot_file)
    gzf = tarfile.open(synphot_file)
    gzf.extractall('./')

### 1.2<font color='white'>-</font>Python Imports<a class="anchor" id="py_imports"></a> ###

In [ ]:
import astropy
from astropy.io import fits, ascii
from astropy.table import Table, QTable
from astropy.nddata import extract_array
from astropy.coordinates import SkyCoord
from astropy import wcs
from astropy.wcs.utils import skycoord_to_pixel
from astropy import units as u
import numpy as np
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astropy.visualization import (simple_norm,LinearStretch)
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import math
from importlib.metadata import version
import pandas as pd

#WebbPSF#
#
import webbpsf

# JWST models
#
from jwst import datamodels, associations
from jwst.datamodels import ImageModel, dqflags

# Background and PSF Functions
#
from photutils.background import MMMBackground, MADStdBackgroundRMS, Background2D, LocalBackground
from photutils.detection import DAOStarFinder
from photutils import EPSFBuilder, GriddedPSFModel
from photutils.psf import DAOGroup, extract_stars, IterativelySubtractedPSFPhotometry, PSFPhotometry

# Photutils library and tools
#
import photutils
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from photutils import Background2D, MedianBackground, ModeEstimatorBackground, MMMBackground

2.<font color='white'>-</font>Download Data<a class="anchor" id="data"></a>
------------------

In [ ]:
# Query the MAST (Mikulski Archive for Space Telescopes) database for observations
# with proposal ID 1028 and a specific filter 'F770W'

obs = Observations.query_criteria(proposal_id=1028, filters=['F770W'])

# Locate a specific observation by its unique observation ID 'jw01537-o024_t001_nircam_clear-f444w-sub160'
#row = obs[obs['obs_id']=='jw01028-o006_t001_miri_f770w']

# Get a list of products associated with the located observation
plist = Observations.get_product_list(obs)

# Filter the product list to include only specific product subgroups: 'RATE', 'CAL', 'I2D', and 'ASN'
fplist = Observations.filter_products(plist, productSubGroupDescription=['CAL', 'I2D', 'ASN'])

# Download the selected products from the MAST database (UNCOMMENT TO DOWNLOAD)
Observations.download_products(fplist)

# Define source and destination directories
source_dir = 'mastDownload/JWST/'
destination_dir = 'mast/01028/'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Use glob to find all files matching the pattern 'mastDownload/JWST/j*/jw01537*cal.fits'
files_to_copy = glob.glob(os.path.join(source_dir, 'j*/jw01028*'))

# Copy the matching files to the destination directory
for file_path in files_to_copy:
    shutil.copy(file_path, destination_dir)


In [ ]:
# Query the MAST (Mikulski Archive for Space Telescopes) database for observations
# with proposal ID 1171 and a specific filters 'F550W' and 'F770W'

obs = Observations.query_criteria(proposal_id=1171, filters=['F560W','F770W'])

# Locate a specific observation by its unique observation ID 'jw01537-o024_t001_nircam_clear-f444w-sub160'
#row = obs[obs['obs_id']=='jw01028-o006_t001_miri_f770w']

# Get a list of products associated with the located observation
plist = Observations.get_product_list(obs)

# Filter the product list to include only specific product subgroups: 'RATE', 'CAL', 'I2D', and 'ASN'
fplist = Observations.filter_products(plist, productSubGroupDescription=['CAL', 'I2D', 'ASN'])
fplist

# Download the selected products from the MAST database (UNCOMMENT TO DOWNLOAD)
Observations.download_products(fplist)

# Define source and destination directories
source_dir = 'mastDownload/JWST/'
destination_dir = 'mast/01171/'

# Create the destination directory if it doesn't exist
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Use glob to find all files matching the pattern 'mastDownload/JWST/j*/jw01537*cal.fits'
files_to_copy = glob.glob(os.path.join(source_dir, 'j*/jw01171*'))

# Copy the matching files to the destination directory
for file_path in files_to_copy:
    shutil.copy(file_path, destination_dir)

3.<font color='white'>-</font>Bright, Single Object<a class="anchor" id="bso"></a>
------------------

The purpose of this section is to illustrate how to perform PSF photometry on a single, bright object. While aperture photometry is feasible in isolated cases, the user may find PSF photometry preferable in crowded fields or complicated backgrounds.

### 3.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="bso2"></a> ###

Generally, PSF photometry for data from a space telescope is most accurately performed on pre-mosaiced data. In the case of HST, that corresponds to FLT files rather than DRZ. And in the case of JWST, this corresponds to Level2 files rather than Level3. The reason is that a mosaiced PSF changes the inherent PSF as a function of position on the detector so that there is no adequate model (theoretical or empirical) to use.<br>

In this example, we aim to fit a source simultaneously across multiple Level 2 images. A more basic approach would be to fit each Level 2 file individually and then average together the measured fluxes. However, this approach more easily corrects for bad pixels or cosmic rays that are only in one image and allows for a more accurate photometric solution by reducing the number of free parameters per source.<br>

Useful references:<br>
HST Documentation on PSF Photometry: https://www.stsci.edu/hst/instrumentation/wfc3/data-analysis/psf<br>
WFPC2 Stellar Photometry with HSTPHOT: https://ui.adsabs.harvard.edu/abs/2000PASP..112.1383D/abstract<br>
Photutils PSF Fitting Photometry: https://photutils.readthedocs.io/en/stable/psf.html

In [ ]:
### Define Level 3 File
lvl3 = ['./mast/01028/jw01028-o006_t001_miri_f770w_i2d.fits']
#lvl3 = ['./mast/01028/obsnum06/jw01028-o006_t001_miri_f770w_i2d.fits']
lvl3

In [ ]:
### Create Level 2 Data List from ASN files

prefix = "./mast/01028/"
asn = glob.glob(prefix+'jw01028-o006_*_image3_00004_asn.json')
with open(asn[0],"r") as fi:
    lvl2 = []
    for ln in fi:
        #print(ln)
        if ln.startswith('                    "expname":'):
            x = ln[2:].split(':')
            y = x[1].split('"')
            lvl2.append(prefix+y[1])
print(lvl2)

In [ ]:
# Examine the First Image (Before DQ Flags Set)
ref_image = lvl2[0]
print(ref_image)

ref_fits = ImageModel(ref_image)
ref_data = ref_fits.data

# The scale should highlight the background noise so it is possible to see all faint sources.
norm1 = simple_norm(ref_data,stretch='log',min_cut=4.5,max_cut=5)

plt.figure(figsize=(20,12))
plt.imshow(ref_data,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.gca().tick_params(axis='both',color='none')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.show()

In [ ]:
# Change all DQ flagged pixels to NANs

# Reference for JWST DQ Flag Definitions: https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html
# In this case, we choose all DQ > 10, but users are encouraged to choose their own values accordingly.
for file in lvl2:
    ref_fits = ImageModel(file)
    data = ref_fits.data
    dq = ref_fits.dq
    data[dq >= 10]=np.nan
    ref_fits.data=data
    ref_fits.save(file)

In [ ]:
# Examine the First Image (After DQ Flags Set)
ref_image = lvl2[0]
print(ref_image)
ref_fits = ImageModel(ref_image)
ref_data = ref_fits.data

# The scale should highlight the background noise so it is possible to see all faint sources.
norm1 = simple_norm(ref_data,stretch='log',min_cut=4.5,max_cut=5)

plt.figure(figsize=(20,12))
plt.imshow(ref_data,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.gca().tick_params(axis='both',color='none')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.show()

In [ ]:
# Zoom in to see the source. In this case, our source is from MIRI Program ID #1028, a Calibration Program.
# We are using Visit 006, which targets the A5V dwarf 2MASSJ17430448+6655015
# Reference Link: http://simbad.cds.unistra.fr/simbad/sim-basic?Ident=2MASSJ17430448%2B6655015&submit=SIMBAD+search

source_location = SkyCoord('17:43:04.4879','+66:55:01.837',unit=(u.hourangle,u.deg))
ref_wcs = ref_fits.get_fits_wcs()
ref_x,ref_y = skycoord_to_pixel(source_location,ref_wcs)
ref_cutout = extract_array(ref_data,(21,21),(ref_y,ref_x))

# The scale should highlight the background noise so it is possible to see all faint sources.
norm1 = simple_norm(ref_cutout,stretch='log',min_cut=4.3,max_cut=205)
plt.imshow(ref_cutout,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.title('PID1028,Obs006')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.gca().tick_params(axis='both',color='none')
plt.show()

#### DEV NOTES: <br> 
1. Add documentation on using webbpsf.calc_psf to have more control over your PSF.<br>

In [ ]:
# Get the PSF from WebbPSF using defaults

source_location = SkyCoord('17:43:04.4879','+66:55:01.837',unit=(u.hourangle,u.deg))

image = lvl2[0]
fits = ImageModel(image)
data = fits.data
err = fits.err
wcs = fits.get_fits_wcs()
x,y = skycoord_to_pixel(source_location,wcs)

# Get the PSF from WebbPSF using defaults
miri = webbpsf.MIRI()
miri.filter =  'F770W'
c = webbpsf.gridded_library.CreatePSFLibrary(miri, filter_name='F770W', num_psfs=4, detectors='all', psf_location = (x,y))
grid = c.create_grid()
#webbpsf.gridded_library.display_psf_grid(grid)

# Convert PSF into an astropy model to be used by photutils
epsf_model = photutils.psf.FittableImageModel(grid.data[0,:,:],normalize=False,oversampling=4)
epsf_model

In [ ]:
# The scale should highlight the background noise so it is possible to see all faint sources.

cutout = epsf_model.data
norm1 = simple_norm(cutout,stretch='log',min_cut=0.0,max_cut=0.01)
plt.imshow(cutout,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.title('WebbPSF Model')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.gca().tick_params(axis='both',color='none')
plt.show()

In [ ]:
# Set up photutils parameters and perform PSF photometry

bkgstat = MMMBackground()
localbkg_estimator = LocalBackground(5, 10, bkgstat)
psf_width = 5
centers = [x,y]
pos = Table(np.atleast_2d(centers),names=['x_0','y_0'])
daofind = DAOStarFinder(threshold=5,fwhm=2,xycoords=np.array([pos['x_0'],pos['y_0']]).T)
psfphot = PSFPhotometry(epsf_model, psf_width, finder=daofind, aperture_radius=psf_width, localbkg_estimator=localbkg_estimator)
phot = psfphot(data, error=err, init_params=pos)
phot

In [ ]:
# Extract Flux and Fluxerr

flux = phot['flux_fit'] # MJy/Str
fluxerr = phot['flux_err'] # MJy/Str

# Convert to magnitudes

units = astropy.units.MJy
magerr = 2.5 * np.log10(1.0 + (fluxerr/flux))
pixel_scale = astropy.wcs.utils.proj_plane_pixel_scales(wcs)[0]  * wcs.wcs.cunit[0].to('arcsec')
flux_units = astropy.units.MJy / astropy.units.sr * (pixel_scale * astropy.units.arcsec)**2
flux = flux*flux_units
fluxerr = fluxerr*flux_units
flux = flux.to(units)
fluxerr = fluxerr.to(units)
mag = flux.to(astropy.units.ABmag)
zp = mag.value+2.5*np.log10(flux.value)
print(mag,magerr)

In [ ]:
# Plot Residual 

resid = psfphot.make_residual_image(data, (21, 21))

# The scale should highlight the background noise so it is possible to see all faint sources.
cutout = extract_array(resid,(25,25),(y,x))
norm1 = simple_norm(cutout,stretch='log',min_cut=-1,max_cut=10)
plt.imshow(cutout,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.title('Residual Image')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.gca().tick_params(axis='both',color='none')
plt.show()

4.<font color='white'>-</font>Faint/Upper Limit, Single Object<a class="anchor" id="fso"></a>
------------------

The purpose of this section is to illustrate how to calculate an upper limit using PSF photometry a blank part of the sky. 

### 4.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="fso2"></a> ###

In [ ]:
### Level 3 Files
lvl3 = ['mast/01028/jw01028-o006_t001_miri_f770w_i2d.fits']
lvl3

In [ ]:
### Create Level 2 Data List from ASN files
prefix = "./mast/01028/"
asn = glob.glob(prefix+'jw01028-o006_*_image3_00004_asn.json')
with open(asn[0],"r") as fi:
    lvl2 = []
    for ln in fi:
        #print(ln)
        if ln.startswith('                    "expname":'):
            x = ln[2:].split(':')
            y = x[1].split('"')
            lvl2.append(prefix+y[1])
print(lvl2)

In [ ]:
# Change all DQ flagged pixels to NANs

# Reference for JWST DQ Flag Definitions: https://jwst-pipeline.readthedocs.io/en/latest/jwst/references_general/references_general.html
# In this case, we choose all DQ > 10, but users are encouraged to choose their own values accordingly.
for file in lvl2:
    ref_fits = ImageModel(file)
    data = ref_fits.data
    dq = ref_fits.dq
    data[dq >= 10]=np.nan
    ref_fits.data=data
    ref_fits.save(file)

In [ ]:
# Examine the First Image (After DQ Flags Set)
ref_image = lvl2[0]
print(ref_image)
ref_fits = ImageModel(ref_image)
ref_data = ref_fits.data

# The scale should highlight the background noise so it is possible to see all faint sources.
norm1 = simple_norm(ref_data,stretch='log',min_cut=4.5,max_cut=5)

plt.figure(figsize=(20,12))
plt.imshow(ref_data,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.gca().tick_params(axis='both',color='none')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.show()

In [ ]:
# Pick a blank part of the sky to calculate the upper limit

source_location = SkyCoord('17:43:00.0332','+66:54:42.677',unit=(u.hourangle,u.deg))
ref_wcs = ref_fits.get_fits_wcs()
ref_x,ref_y = skycoord_to_pixel(source_location,ref_wcs)
ref_cutout = extract_array(ref_data,(21,21),(ref_y,ref_x))

# The scale should highlight the background noise so it is possible to see all faint sources.
norm1 = simple_norm(ref_cutout,stretch='log',min_cut=4.5,max_cut=5)
plt.imshow(ref_cutout,origin='lower',norm=norm1,cmap='gray')
clb = plt.colorbar()
clb.set_label('MJy/Str', labelpad=-40, y=1.05, rotation=0)
plt.title('PID1028,Obs006')
plt.xlabel('Pixels')
plt.ylabel('Pixels')
plt.gca().tick_params(axis='both',color='none')
plt.show()

In [ ]:
# Get the PSF from WebbPSF using defaults

image = lvl2[0]
fits = ImageModel(image)
data = fits.data
err = fits.err
wcs = fits.get_fits_wcs()
x,y = skycoord_to_pixel(source_location,wcs)

# Get the PSF from WebbPSF using defaults
miri = webbpsf.MIRI()
miri.filter =  'F770W'
c = webbpsf.gridded_library.CreatePSFLibrary(miri, filter_name='F770W', num_psfs=4, detectors='all', psf_location = (x,y))
grid = c.create_grid()
#webbpsf.gridded_library.display_psf_grid(grid)

# Convert PSF into an astropy model to be used by photutils
epsf_model = photutils.psf.FittableImageModel(grid.data[0,:,:],normalize=False,oversampling=4)
epsf_model

In [ ]:
# Set up photutils parameters and perform PSF photometry

bkgstat = MMMBackground()
localbkg_estimator = LocalBackground(5, 10, bkgstat)
psf_width = 5
centers = [x,y]
pos = Table(np.atleast_2d(centers),names=['x_0','y_0'])
daofind = DAOStarFinder(threshold=5,fwhm=2,xycoords=np.array([pos['x_0'],pos['y_0']]).T)
psfphot = PSFPhotometry(epsf_model, psf_width, finder=daofind, aperture_radius=psf_width, localbkg_estimator=localbkg_estimator)
phot = psfphot(data, error=err, init_params=pos)
phot

In [ ]:
# Extract Flux and Fluxerr (and calculate the upper limit from the error)

flux = phot['flux_fit'] # MJy/Str
fluxerr = phot['flux_err'] # MJy/Str

# To calculate upper limit, take your fluxerr and multiply by your defined sigma
sigma = 3
flux = fluxerr*sigma 

# Convert to magnitudes

units = astropy.units.MJy
magerr = 2.5 * np.log10(1.0 + (fluxerr/flux))
pixel_scale = astropy.wcs.utils.proj_plane_pixel_scales(wcs)[0]  * wcs.wcs.cunit[0].to('arcsec')
flux_units = astropy.units.MJy / astropy.units.sr * (pixel_scale * astropy.units.arcsec)**2
flux = flux*flux_units
fluxerr = fluxerr*flux_units
flux = flux.to(units)
fluxerr = fluxerr.to(units)
mag = flux.to(astropy.units.ABmag)
zp = mag.value+2.5*np.log10(flux.value)
print(mag)

## Note you can go significantly deeper with the Level3 combined data product

5.<font color='white'>-</font>Stellar Field (LMC)<a class="anchor" id="lmc"></a>
------------------

#### In this case, we are going to do the same steps as in Section 3, but for multiple stars. The purpose is to illustrate the workflow and runtime for using space_phot on a large number of stars. We suggest that space_phot may be less optimal for large numbers of bright stars. Other programs, such as DOLPHOT or Photutils, may be better suited for this use case. The primary advantage to space_phot is on faint, single sources. But it can be extended to a larger number if desired.

### 5.1<font color='white'>-</font>Multiple, Level2 Files<a class="anchor" id="lmc2"></a> ###

##### Now do the same thing for a larger group of stars and test for speed

In [ ]:
### Level 3 Files
#lvl3 = ['mastDownload/JWST/jw01171-o004_t001_miri_f560w/jw01171-o004_t001_miri_f560w_i2d.fits']
lvl3 = ["./mast/01171/jw01171-o004_t001_miri_f560w_i2d.fits"]
lvl3

In [ ]:
### Level 2 Files
#lvl2 = glob.glob('mastDownload/JWST/jw01171004*/*cal.fits')

lvl2 = glob.glob('./mast/01171/jw01171004*cal.fits')
lvl2

In [ ]:
# Find Stars in Level 3 File

# Get rough estimate of background (There are Better Ways to Do Background Subtraction)
bkgrms = MADStdBackgroundRMS()
mmm_bkg = MMMBackground()

#im = fits.open(lvl3[0]) 
#w = wcs.WCS(im['SCI',1])

ref_fits = ImageModel(lvl3[0])
w = ref_fits.get_fits_wcs()

#std = bkgrms(im[1].data)
#bkg = mmm_bkg(im[1].data)
#data_bkgsub = im[1].data.copy()

std = bkgrms(ref_fits.data)
bkg = mmm_bkg(ref_fits.data)
data_bkgsub = ref_fits.data.copy()
data_bkgsub -= bkg        
sigma_psf = 1.636 #pixls for F770W
threshold = 5.

daofind = DAOStarFinder(threshold=threshold * std, fwhm=sigma_psf, exclude_border=True)
found_stars = daofind(data_bkgsub)

In [ ]:
found_stars.pprint_all(max_lines=10)

In [ ]:
# Filter out only stars you want

plt.figure(figsize=(12, 8))
plt.clf()

ax1 = plt.subplot(2, 1, 1)

ax1.set_xlabel('mag')
ax1.set_ylabel('sharpness')

xlim0 = np.min(found_stars['mag']) - 0.25
xlim1 = np.max(found_stars['mag']) + 0.25
ylim0 = np.min(found_stars['sharpness']) - 0.15
ylim1 = np.max(found_stars['sharpness']) + 0.15

ax1.set_xlim(xlim0, xlim1)
ax1.set_ylim(ylim0, ylim1)

#ax1.xaxis.set_major_locator(ticker.AutoLocator())
#ax1.xaxis.set_minor_locator(ticker.AutoMinorLocator())
#ax1.yaxis.set_major_locator(ticker.AutoLocator())
#ax1.yaxis.set_minor_locator(ticker.AutoMinorLocator())

ax1.scatter(found_stars['mag'], found_stars['sharpness'], s=10, color='k')

sh_inf = 0.40
sh_sup = 0.82
#mag_lim = -5.0
lmag_lim = -3.0
umag_lim = -5.0

ax1.plot([xlim0, xlim1], [sh_sup, sh_sup], color='r', lw=3, ls='--')
ax1.plot([xlim0, xlim1], [sh_inf, sh_inf], color='r', lw=3, ls='--')
ax1.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')
ax1.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')

ax2 = plt.subplot(2, 1, 2)

ax2.set_xlabel('mag')
ax2.set_ylabel('roundness')

ylim0 = np.min(found_stars['roundness2']) - 0.25
ylim1 = np.max(found_stars['roundness2']) - 0.25

ax2.set_xlim(xlim0, xlim1)
ax2.set_ylim(ylim0, ylim1)

#ax2.xaxis.set_major_locator(ticker.AutoLocator())
#ax2.xaxis.set_minor_locator(ticker.AutoMinorLocator())
#ax2.yaxis.set_major_locator(ticker.AutoLocator())
#ax2.yaxis.set_minor_locator(ticker.AutoMinorLocator())

round_inf = -0.40
round_sup = 0.40

ax2.scatter(found_stars['mag'], found_stars['roundness2'], s=10, color='k')

ax2.plot([xlim0, xlim1], [round_sup, round_sup], color='r', lw=3, ls='--')
ax2.plot([xlim0, xlim1], [round_inf, round_inf], color='r', lw=3, ls='--')
ax2.plot([lmag_lim, lmag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')
ax2.plot([umag_lim, umag_lim], [ylim0, ylim1], color='r', lw=3, ls='--')

plt.tight_layout()

In [ ]:
mask = ((found_stars['mag'] < lmag_lim) & (found_stars['mag'] > umag_lim) & (found_stars['roundness2'] > round_inf)
        & (found_stars['roundness2'] < round_sup) & (found_stars['sharpness'] > sh_inf) 
        & (found_stars['sharpness'] < sh_sup) & (found_stars['xcentroid'] > 100) & (found_stars['xcentroid'] < 700)
        & (found_stars['ycentroid'] > 100) & (found_stars['ycentroid'] < 700))

found_stars_sel = found_stars[mask]

print('Number of stars found originally:', len(found_stars))
print('Number of stars in final selection:', len(found_stars_sel))


In [ ]:
found_stars_sel

In [ ]:
# Convert pixel to wcs coords
from astropy.wcs.utils import skycoord_to_pixel
skycoords = w.pixel_to_world(found_stars_sel['xcentroid'], found_stars_sel['ycentroid'])
skycoords = skycoords[9:] #Cutting down the list artificially to make things run faster
len(skycoords)

In [ ]:
# Change all DQ flagged pixels to NANs
for file in lvl2:
    #hdul = fits.open(file, mode='update')
    #data = fits.open(file)['SCI',1].data
    #dq = fits.open(file)['DQ',1].data
    #data[dq == 262657]=np.nan
    #data[dq == 262661]=np.nan
    #hdul['SCI',1].data=data
    #hdul.flush()

    ref_fits = ImageModel(file)
    data = ref_fits.data
    dq = ref_fits.dq
    data[dq >= 10]=np.nan
    ref_fits.data=data
    ref_fits.save(file)

In [ ]:
t = QTable([skycoords], names=["skycoord"])
t.write('skycoord.ecsv',overwrite=True)

In [ ]:
# Make a single generic PSF for now
miri = webbpsf.MIRI()
miri.filter =  'F770W'
c = webbpsf.gridded_library.CreatePSFLibrary(miri, filter_name='F770W', num_psfs=4, detectors='all')
grid = c.create_grid()

# Convert PSF into an astropy model to be used by photutils
epsf_model = photutils.psf.FittableImageModel(grid.data[0,:,:],normalize=False,oversampling=4)
epsf_model

In [ ]:
# Get a list of x,y coords for each star in the list
image = lvl2[0]
fits = ImageModel(image)
data = fits.data
err = fits.err
wcs = fits.get_fits_wcs()
x,y = skycoord_to_pixel(skycoords,wcs)
centers = []
for i in range(0,len(x)):
    center = [x[i],y[i]]
    centers.append(center)

centers

In [ ]:
# Build Photometry Table

bkgstat = MMMBackground()
localbkg_estimator = LocalBackground(5, 10, bkgstat)
psf_width = 5
pos = Table(np.atleast_2d(centers),names=['x_0','y_0'])
daofind = DAOStarFinder(threshold=5,fwhm=2,xycoords=np.array([pos['x_0'],pos['y_0']]).T)
psfphot = PSFPhotometry(epsf_model, psf_width, finder=daofind, aperture_radius=psf_width, localbkg_estimator=localbkg_estimator)
phot = psfphot(data, error=err, init_params=pos)
phot

In [ ]:
# Extract Flux and Fluxerr (and calculate the upper limit from the error)

flux = phot['flux_fit'] # MJy/Str
fluxerr = phot['flux_err'] # MJy/Str

# Convert to magnitudes

units = astropy.units.MJy
magerr = 2.5 * np.log10(1.0 + (fluxerr/flux))
pixel_scale = astropy.wcs.utils.proj_plane_pixel_scales(wcs)[0]  * wcs.wcs.cunit[0].to('arcsec')
flux_units = astropy.units.MJy / astropy.units.sr * (pixel_scale * astropy.units.arcsec)**2
flux = flux*flux_units
fluxerr = fluxerr*flux_units
flux = flux.to(units)
fluxerr = fluxerr.to(units)
mag = flux.to(astropy.units.ABmag)
zp = mag.value+2.5*np.log10(flux.value)
print(mag,magerr)

<hr style="border:1px solid gray"> </hr>

<img style="float: center;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/>